# Intermediate Machine Learning
## Pipelines

- **Pipelines** are a simple way to keep your data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so you can use the whole bundle as if it were a single step.

Benefits:
- Cleaner Code
- Fewer Bugs
- Easier to Productionize
- More options for model validation

Tutorial: https://www.kaggle.com/code/alexisbcook/pipelines 

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('melbourne_housing_snapshot/melb_data.csv')
print(data.shape)
data.sample(5, random_state=0)

(13580, 21)


Suburb          Address  Rooms Type      Price Method  \
8505     Williamstown    44 Electra St      4    h  2165000.0     SP   
5523           Seddon      80 Gamon St      2    h   815000.0      S   
12852  Sunshine North      6 Melton Av      3    h   610000.0     SP   
4818          Prahran       16 Park Rd      3    t  1245000.0     PI   
12812     Pascoe Vale  13 Yorkshire St      3    h  1160000.0      S   

        SellerG        Date  Distance  Postcode  ...  Bathroom  Car  Landsize  \
8505       Greg   6/05/2017       8.0    3016.0  ...       2.0  2.0     450.0   
5523   Chisholm  30/07/2016       6.6    3011.0  ...       1.0  0.0     172.0   
12852   Sweeney  16/09/2017      10.5    3020.0  ...       1.0  1.0     581.0   
4818   Marshall   6/08/2016       4.5    3181.0  ...       2.0  1.0     128.0   
12812    Nelson  16/09/2017       8.5    3044.0  ...       2.0  2.0     480.0   

       BuildingArea  YearBuilt  CouncilArea Lattitude  Longtitude  \
8505          190.0     1910.0  Hobsons Bay -37.86100   144.89850   
5523           81.0     1900.0  Maribyrnong -37.81000   144.88960   
12852           NaN        NaN          NaN -37.76740   144.82421   
4818          134.0     2000.0  Stonnington -37.85260   145.00710   
12812           NaN        NaN          NaN -37.72523   144.94567   

                  Regionname Propertycount  
8505    Western Metropolitan        6380.0  
5523    Western Metropolitan        2417.0  
12852   Western Metropolitan        4217.0  
4818   Southern Metropolitan        7717.0  
12812  Northern Metropolitan        7485.0  

[5 rows x 21 columns]

In [9]:
# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

print(X_train_full.shape, X_valid_full.shape)
print(y_train.shape, y_valid.shape)

(10864, 20) (2716, 20)
(10864,) (2716,)


In [11]:
X_train_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10864 entries, 12167 to 2732
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         10864 non-null  object 
 1   Address        10864 non-null  object 
 2   Rooms          10864 non-null  int64  
 3   Type           10864 non-null  object 
 4   Method         10864 non-null  object 
 5   SellerG        10864 non-null  object 
 6   Date           10864 non-null  object 
 7   Distance       10864 non-null  float64
 8   Postcode       10864 non-null  float64
 9   Bedroom2       10864 non-null  float64
 10  Bathroom       10864 non-null  float64
 11  Car            10815 non-null  float64
 12  Landsize       10864 non-null  float64
 13  BuildingArea   5708 non-null   float64
 14  YearBuilt      6557 non-null   float64
 15  CouncilArea    9792 non-null   object 
 16  Lattitude      10864 non-null  float64
 17  Longtitude     10864 non-null  float64
 18  Reg

Keeping only the low_cardinality columns and the columns that contains numerical values.

In [10]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10864 entries, 12167 to 2732
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Type           10864 non-null  object 
 1   Method         10864 non-null  object 
 2   Regionname     10864 non-null  object 
 3   Rooms          10864 non-null  int64  
 4   Distance       10864 non-null  float64
 5   Postcode       10864 non-null  float64
 6   Bedroom2       10864 non-null  float64
 7   Bathroom       10864 non-null  float64
 8   Car            10815 non-null  float64
 9   Landsize       10864 non-null  float64
 10  BuildingArea   5708 non-null   float64
 11  YearBuilt      6557 non-null   float64
 12  Lattitude      10864 non-null  float64
 13  Longtitude     10864 non-null  float64
 14  Propertycount  10864 non-null  float64
dtypes: float64(11), int64(1), object(3)
memory usage: 1.3+ MB


In [13]:
X_train.head()

Type Method             Regionname  Rooms  Distance  Postcode  Bedroom2  \
12167    u      S  Southern Metropolitan      1       5.0    3182.0       1.0   
6524     h     SA   Western Metropolitan      2       8.0    3016.0       2.0   
8413     h      S   Western Metropolitan      3      12.6    3020.0       3.0   
2919     u     SP  Northern Metropolitan      3      13.0    3046.0       3.0   
6043     h      S   Western Metropolitan      3      13.3    3020.0       3.0   

       Bathroom  Car  Landsize  BuildingArea  YearBuilt  Lattitude  \
12167       1.0  1.0       0.0           NaN     1940.0  -37.85984   
6524        2.0  1.0     193.0           NaN        NaN  -37.85800   
8413        1.0  1.0     555.0           NaN        NaN  -37.79880   
2919        1.0  1.0     265.0           NaN     1995.0  -37.70830   
6043        1.0  2.0     673.0         673.0     1970.0  -37.76230   

       Longtitude  Propertycount  
12167    144.9867        13240.0  
6524     144.9005         6380.0  
8413     144.8220         3755.0  
2919     144.9158         8870.0  
6043     144.8272         4217.0